In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("asludds")

from lmphoton.simulation import current_simulation as sim
import lmtimesim
from lmphoton import OptNetwork
from lmphoton.models import DirectionalCoupler
from lmtimesim.components.Filters.crr_phase_shifter import CRR_PhaseShifter
from tqdm import tqdm

from scipy.constants import c

um = 1e-6

# Define a custom CRR component with tunable coupling rates
I took the CRR code from the following link and lightly modified it to expose the coupling rates externally:
https://github.com/lightmatter-ai/lmtimesim/blob/main/lmtimesim/components/Filters/crr.py

In [ ]:
class CRR(OptNetwork):

    def __init__(
            self,
            ports=('1l', '2r', '3r', '4l'),
            name='CRR',
            dL=0, #Length offset to CRR waveguides
            crr_index=1,
            radius=4.465 * um,
            bus_power_coupling = 0.2809,
            ring_ring_power_coupling = 0.0484,
            phase_shifter_class=CRR_PhaseShifter):
        ## Device physical parameters
        self.name = name
        self.crr_index = crr_index
        self.waveguide_loss_rate = 1.3
        self.radius = radius
        self.lc = 5 * um
        #L_waveguide is the round trip length of the CRR
        self.L_waveguide = 2 * np.pi * self.radius + 2 * self.lc + 2 * dL #dL added to both sides of the ring
        self.external_coupling = bus_power_coupling
        self.internal_coupling = ring_ring_power_coupling

        #Thermo-optic properties
        self._bottom_ring_heater_voltage = 0
        self._top_ring_heater_voltage = 0

        self._rc_filter_bool = False

        self.ps1 = phase_shifter_class(length=self.L_waveguide, loss_rate=self.waveguide_loss_rate)
        self.ps2 = phase_shifter_class(length=self.L_waveguide, loss_rate=self.waveguide_loss_rate)

        self.dc_bus = DirectionalCoupler(splitratio=self.external_coupling)
        self.dc_internal = DirectionalCoupler(splitratio=self.internal_coupling)
        self.dc_drop = DirectionalCoupler(splitratio=self.external_coupling)

        super().__init__(
            ports=ports,
            name=name,
            children={
                f'DC1_CRR_{self.crr_index}': self.dc_bus, #Bus port cavity
                f'PS1_CRR_{self.crr_index}':
                self.ps1, #Phase shifter is a standing for thermal tuner
                f'DC2_CRR_{self.crr_index}': self.dc_internal, #Internal coupler
                f'PS2_CRR_{self.crr_index}': self.ps2,
                f'DC3_CRR_{self.crr_index}': self.dc_drop, # Drop port cavity
            },
            links=[
                (f'DC1_CRR_{self.crr_index}:PORT3', f'PS1_CRR_{self.crr_index}:PORT1'),
                (f'PS1_CRR_{self.crr_index}:PORT2', f'DC2_CRR_{self.crr_index}:PORT1'),
                (f'DC2_CRR_{self.crr_index}:PORT2', f'DC1_CRR_{self.crr_index}:PORT4'),
                (f'DC2_CRR_{self.crr_index}:PORT4', f'PS2_CRR_{self.crr_index}:PORT1'),
                (f'PS2_CRR_{self.crr_index}:PORT2', f'DC3_CRR_{self.crr_index}:PORT1'),
                (f'DC3_CRR_{self.crr_index}:PORT2', f'DC2_CRR_{self.crr_index}:PORT3'),
            ])

    @property
    def bottom_ring_heater_voltage(self):
        return self.ps1.heater_voltage

    @bottom_ring_heater_voltage.setter
    def bottom_ring_heater_voltage(self, new_voltage):
        self.ps1.heater_voltage = new_voltage

    @property
    def top_ring_heater_voltage(self):
        return self.ps2.heater_voltage

    @top_ring_heater_voltage.setter
    def top_ring_heater_voltage(self, new_voltage):
        self.ps2.heater_voltage = new_voltage

    @property
    def bottom_ring_heater_current(self):
        return self.ps1.heater_current

    @property
    def top_ring_heater_current(self):
        return self.ps2.heater_current

    @property
    def rc_filter_bool(self):
        return self._rc_filter_bool

    @rc_filter_bool.setter
    def rc_filter_bool(self, new_rc_filter_bool):
        self._rc_filter_bool = new_rc_filter_bool
        self.ps1.rc_filter_bool = self.rc_filter_bool
        self.ps2.rc_filter_bool = self.rc_filter_bool

    def thru_port_transmission(self):
        return np.abs(self._construct_smatrix()[1, 0])**2

    def drop_port_transmission(self):
        return np.abs(self._construct_smatrix()[3, 0])**2

    def check_children_timesteps(self):
        if (self.ps1.check_children_timesteps()) and (self.ps2.check_children_timesteps()):
            return True
        else:
            return False
        

class CustomCRR(CRR):

    class CustomCRRPhaseShifter(CRR_PhaseShifter):

        def __init__(self,
                     length,
                     loss_rate,
                     ports: tuple[str, str] = ('1l', '2r'),
                     name: str = 'PS'):
            index = 2.45        # Updated from LMPDK
            group_index = 4.276 # Updated from LMPDK
            thermal_time_constant1 = 26e-6
            thermal_time_constant2 = 120e-6
            thermal_weight1 = 0.52
            thermal_weight2 = 0.48
            Rheat0 = 2 * 26.1
            rheat_tc = 0.001626
            waveguide_thermal_resistance = 2 / 3 * 2 * 9834 #Degrees C per Watt
            heater_thermal_resistance = 2 * 9788.8 #Degrees C per Watt
            #2/3 comes from thermal crosstalk model to make sure this model is valid
            

            super().__init__(ports=ports,
                             name=name,
                             length=length,
                             index=index,
                             group_index=group_index,
                             loss_rate=loss_rate,
                             thermal_time_constant1=thermal_time_constant1,
                             thermal_time_constant2=thermal_time_constant2,
                             thermal_weight1=thermal_weight1,
                             thermal_weight2=thermal_weight2,
                             Rheat0=Rheat0,
                             rheat_tc=rheat_tc,
                             waveguide_thermal_resistance=waveguide_thermal_resistance,
                             heater_thermal_resistance=heater_thermal_resistance)

    #A CRR for razorcrest designed for operation on 1800GHz FSR
    def __init__(
            self,
            ports=('1l', '2r', '3r', '4l'),
            name='Custom_CRR',
            dL=0, #Length offset to CRR waveguides
            bus_power_coupling = 0.2809,
            ring_ring_power_coupling = 0.0484,
            crr_index=1):
        super().__init__(ports=ports,
                         name=name,
                         dL=dL,
                         crr_index=crr_index,
                         radius=4.465 * um,
                         bus_power_coupling = bus_power_coupling,
                         ring_ring_power_coupling = ring_ring_power_coupling,
                         phase_shifter_class=CustomCRR.CustomCRRPhaseShifter)

In [ ]:
#Pairs of coupling rates targetting different design points
coupling_rate_pairs = [
    (0.2809,0.0484), #index=0, for ~142GHz 1dB passband
    (0.241,0.035),   #index=1, for ~120GHz 1dB passband
    (0.225,0.0291),  #index=2, for ~110GHz 1dB passband
    (0.21,0.0245),   #index=3, for ~100GHz 1dB passband
    (0.185,0.0195),  #index=4, for ~90GHz 1dB passband
]

crr_index = 4

bus_power_coupling, ring_ring_power_coupling = coupling_rate_pairs[crr_index]
crr = CustomCRR(
    bus_power_coupling = bus_power_coupling,
    ring_ring_power_coupling = ring_ring_power_coupling,
)

# Do broad wavelength sweeps of the CRR
wavelengths = np.linspace(1.308, 1.312, 10000)*um

for i in range(10):
    crr.bottom_ring_heater_voltage = 0.7130930930930931 #Voltage found below to park centered at 1310nm
    crr.top_ring_heater_voltage = 0.7130930930930931
    crr.rc_filter_bool = False
    crr.drop_port_transmission()
drop_port_storage = []
thru_port_storage = []

for wavelength in wavelengths:
    sim().wavelength = wavelength
    crr._wavelength = wavelength
    drop_port_storage.append(crr.drop_port_transmission())
    thru_port_storage.append(crr.thru_port_transmission())

drop_port_storage = np.array(drop_port_storage)
thru_port_storage = np.array(thru_port_storage)

with open(f'storage/crr_{crr_index}.npy', 'wb') as f:
    np.save(f, wavelengths)
    np.save(f, drop_port_storage)

#From the response, what is the ripple? What is the 1dB bandwidth?
sim().wavelength = 1310e-9
crr._wavelength  = 1310e-9
print(f"Ripple (dB) = {10*np.log10(crr.drop_port_transmission()/np.max(drop_port_storage))}")

def find_crossings(x, y, target_value):
    # Find indices where the curve crosses the target value
    indices = np.where(np.diff(np.sign(y - target_value)))[0]
    
    # Initialize list to store crossing points
    crossing_points = []

    for idx in indices:
        # Points around the crossing
        x1, x2 = x[idx], x[idx + 1]
        y1, y2 = y[idx], y[idx + 1]
        
        # Linear interpolation
        if y2 != y1:  # Avoid division by zero
            x_cross = x1 + (target_value - y1) * (x2 - x1) / (y2 - y1)
            crossing_points.append(x_cross)
        else:
            # If y1 == y2 and it's exactly the target value, both points are crossings
            if y1 == target_value:
                crossing_points.extend([x1, x2])
    
    return np.array(crossing_points)

power_intersection_1dB_down_from_max = np.max(drop_port_storage)*10**(-1/10)

crossing_points = find_crossings(wavelengths,drop_port_storage,power_intersection_1dB_down_from_max)

print(crossing_points)
print(f"Bandwidth 1dB: {np.abs(c/crossing_points[0] - c/crossing_points[-1]) *1e-9} (GHz)")

plt.figure()
plt.plot(wavelengths*1e9,drop_port_storage)
plt.plot(wavelengths*1e9,thru_port_storage)
plt.vlines(x=crossing_points[0]*1e9, ymin=0,ymax=1,linestyle='--',color='grey')
plt.vlines(x=crossing_points[-1]*1e9,ymin=0,ymax=1,linestyle='--',color='grey')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Transmission")
plt.legend(["Drop Port","Thru Port"])
plt.show()


| Device index | 1dB bandwidth | Ripple (dB) |
|--------------|---------------|-------------|
| 0            | 142           | 0.38        |
| 1            | 120           | 0.41        |
| 2            | 110           | 0.37        |
| 3            | 100           | 0.35        |
| 4            | 90            | 0.41        |

In [ ]:
plt.figure()
for crr_index in range(0,5):
    with open(f'storage/crr_{crr_index}.npy', 'rb') as f:
        wav = np.load(f)
        drop = np.load(f)
        plt.plot(wav*1e9,drop,label=f'CRR {crr_index}')
plt.legend()
plt.xlabel("Wavelength (nm)")
plt.ylabel("Transmission")
plt.show()

plt.figure()
for crr_index in range(0,5):
    with open(f'storage/crr_{crr_index}.npy', 'rb') as f:
        wav = np.load(f)
        drop = np.load(f)
        plt.plot(wav*1e9,10*np.log10(drop),label=f'CRR {crr_index}')
plt.legend()
plt.xlabel("Wavelength (nm)")
plt.ylabel("Transmission (dB)")
plt.show()

In [ ]:
# Do broad wavelength sweeps of the CRR
wavelengths = np.linspace(1.298, 1.312, 10000)*um

crr.bottom_ring_heater_voltage = 0
crr.top_ring_heater_voltage = 0
crr.rc_filter_bool = False
drop_port_storage = []
thru_port_storage = []

for wavelength in wavelengths:
    sim().wavelength = wavelength
    crr._wavelength = wavelength
    drop_port_storage.append(crr.drop_port_transmission())

plt.figure()
plt.plot(wavelengths*1e9,drop_port_storage)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Transmission")
plt.legend(["Drop Port","Thru Port"])
plt.show()

In [ ]:
# Do broad wavelength sweeps of the CRR
wavelengths = np.linspace(1.298, 1.312, 10000)*um

crr.bottom_ring_heater_voltage = 0
crr.top_ring_heater_voltage = 0
crr.rc_filter_bool = False
drop_port_storage = []
thru_port_storage = []

for wavelength in wavelengths:
    sim().wavelength = wavelength
    crr._wavelength = wavelength
    drop_port_storage.append(crr.drop_port_transmission())
    thru_port_storage.append(crr.thru_port_transmission())

plt.figure()
plt.plot(wavelengths*1e9,drop_port_storage)
plt.plot(wavelengths*1e9,thru_port_storage)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Transmission")
plt.legend(["Drop Port","Thru Port"])
plt.show()

In [ ]:
# Align the CRR to 1310nm center wavelength
heater_voltage_sweep = np.linspace(0.705,0.72,num=1000)
drop_port_storage = []

sim().wavelength = 1310e-9
crr._wavelength = 1310e-9

crr.bottom_ring_heater_voltage = np.min(heater_voltage_sweep)
crr.top_ring_heater_voltage = np.min(heater_voltage_sweep)
crr.drop_port_transmission()

for v in heater_voltage_sweep:
    crr.bottom_ring_heater_voltage = v
    crr.top_ring_heater_voltage = v
    drop_port_storage.append(crr.drop_port_transmission())
    
drop_port_storage = np.array(drop_port_storage)

plt.figure()
plt.plot(heater_voltage_sweep,drop_port_storage)
plt.xlabel("Heater voltage (V)")
plt.ylabel("Transmission")
plt.show()

voltage_to_park_at_center = heater_voltage_sweep[np.argmin(drop_port_storage)]
print(f"Middle of dip voltage {voltage_to_park_at_center}")

crr.bottom_ring_heater_voltage = voltage_to_park_at_center
crr.top_ring_heater_voltage = voltage_to_park_at_center

In [ ]:
# Create a 56G NRZ spectrum
def generate_56G_spectrum(center_wavelength,wavelength_sweep):
    frequency_sweep = c/wavelength_sweep
    center_frequency = c/center_wavelength
    diff_freq = center_frequency - frequency_sweep
    Tb = 1/(56e9)
    tp = np.array([0.5 * Tb * (np.sin(np.pi*f*Tb)/(np.pi*f*Tb))**2 for f in diff_freq])

    tp /= np.max(tp)

    return tp

def raised_cosine(center_wavelength,wavelength_sweep,T=1/(56e9),beta=0.28):
    frequency_sweep = c/wavelength_sweep
    center_frequency = c/center_wavelength
    diff_freq = center_frequency - frequency_sweep
    returnable = []
    for f in diff_freq:
        if np.abs(f) < (1-beta)/(2*T):
            returnable.append(1)
        elif ((1-beta)/(2*T) < np.abs(f)) and (np.abs(f) < (1+beta)/(2*T)):
            returnable.append(0.5 * (1 + np.cos(np.pi *T /beta * (np.abs(f) - (1-beta)/(2*T)))))
        else:
            returnable.append(0)
    return np.array(returnable)

# Applying the raised cosine filter to data:
# rc = raised_cosine(freqs[idx],1/data_rate,beta=0.28)

from scipy.constants import c
baseline_wavelength = 1310e-9
baseline_frequency = c/baseline_wavelength
baseline_frequency_plus_50GHz = baseline_frequency + 50e9
baseline_wavelength_plus_50GHz = c/baseline_frequency_plus_50GHz
baseline_frequency_plus_150GHz = baseline_frequency + 150e9
baseline_wavelength_plus_150GHz = c/baseline_frequency_plus_150GHz

tp_50GHz = raised_cosine(baseline_wavelength_plus_50GHz,wavelengths)
tp_150GHz = raised_cosine(baseline_wavelength_plus_150GHz,wavelengths)

#Create a CRR spectrum to compare
drop_port_storage = []

for wavelength in wavelengths:
    sim().wavelength = wavelength
    crr._wavelength = wavelength
    drop_port_storage.append(crr.drop_port_transmission())

drop_port_storage = np.array(drop_port_storage)

plt.figure()
plt.plot(wavelengths*1e9,tp_50GHz)
plt.plot(wavelengths*1e9,tp_150GHz)
plt.plot(wavelengths*1e9,drop_port_storage)
plt.xlim(1308,1312)
tick_locations = [1308,1308.5,1309,1309.5,1310,1310.5,1311,1311.5,1312]
plt.xticks(tick_locations,[str(i) for i in tick_locations])
plt.xlabel("Wavelength (nm)")
plt.ylabel("Amplitude")
plt.show()

# Plot the effect of attenuation
plt.figure()
plt.plot(wavelengths*1e9,tp_50GHz*drop_port_storage)
plt.plot(wavelengths*1e9,tp_150GHz*drop_port_storage)
plt.plot(wavelengths*1e9,drop_port_storage)
plt.xlim(1308,1312)
tick_locations = [1308,1308.5,1309,1309.5,1310,1310.5,1311,1311.5,1312]
plt.xticks(tick_locations,[str(i) for i in tick_locations])
plt.xlabel("Wavelength (nm)")
plt.ylabel("Amplitude")
plt.show()

# Measure the sum of the spectrum and compare their ratios
ratio = np.sum(tp_50GHz*drop_port_storage)/np.sum(tp_150GHz*drop_port_storage)
print(ratio)

In [ ]:
#Create a monte-carlo sim to run
def monte_carlo_sim():
    baseline_wavelength = 1310e-9
    baseline_frequency = c/baseline_wavelength
    baseline_freq_with_variation = np.random.normal(baseline_frequency,scale=(40e9)/4) #Factor of 4 because of 4-sigma
    baseline_freq_aggressor_with_variation = np.random.normal(baseline_frequency+200e9,scale=(40e9)/4)
    baseline_wavelength_with_variation = c/baseline_freq_with_variation
    baseline_wavelength_aggressor_with_variation = c/baseline_freq_aggressor_with_variation

    tp_main = raised_cosine(baseline_wavelength_with_variation,wavelengths)
    tp_aggressor = raised_cosine(baseline_wavelength_aggressor_with_variation,wavelengths)

    # Measure the sum of the spectrum and compare their ratios
    delta_frequency = baseline_freq_aggressor_with_variation - baseline_freq_with_variation
    ratio = np.sum(tp_main*drop_port_storage)/np.sum(tp_aggressor*drop_port_storage)
    return delta_frequency, ratio

monte_carlo_freq_storage = []
monte_carlo_ratio_storage = []
for i in tqdm(range(5000),desc='Monte carlo sim'):
    delta_frequency, ratio = monte_carlo_sim()
    monte_carlo_freq_storage.append(delta_frequency)
    monte_carlo_ratio_storage.append(ratio)

monte_carlo_freq_storage = np.array(monte_carlo_freq_storage)
monte_carlo_ratio_storage = np.array(monte_carlo_ratio_storage)

In [ ]:
plt.figure()
plt.hist(monte_carlo_freq_storage*1e-9,bins=100)
plt.xlabel("Frequency Offset Between Channels (GHz)")
plt.show()

plt.figure()
plt.hist(monte_carlo_ratio_storage  ,bins=100)
plt.xlabel("Ratio of desired power to aggressor")
plt.show()

plt.figure()
plt.ecdf(monte_carlo_freq_storage*1e-9  ,complementary=True)
plt.hlines(0.9995,xmin=125,xmax=200,color='grey',linestyle='--')
plt.xlim(130,150)
plt.ylim(0.999,1)
plt.legend(['CDF',r'$4\sigma$'])
plt.title("Frequency difference CDF")
plt.xlabel("Frequency difference (GHz)")
plt.show()

plt.figure()
plt.ecdf(monte_carlo_ratio_storage  ,complementary=True)
plt.hlines(0.9995,xmin=5,xmax=20,color='grey',linestyle='--')
plt.xlim(6.5,10)
plt.ylim(0.999,1)
plt.legend(['CDF',r'$4\sigma$'])
plt.title("Ratio CDF")
plt.xlabel("Ratio")
plt.show()

print(np.percentile(monte_carlo_ratio_storage,100*(1-0.9995)))
print(np.percentile(monte_carlo_freq_storage*1e-9,100*(1-0.9995)))
